In [1]:
# ==============================================================================
# SENTINELA FLUVIAL - PIPELINE DE INFERÊNCIA COM MEDGEMMA 4B
# ==============================================================================
# Instalação de dependências para rodar em 4-bit (memória eficiente)
!pip install -q transformers

print("✅ All packages installed!")

✅ All packages installed!


In [2]:
import json

from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("HF_TOKEN")

login(token=secret_value_0)

print("✅ Logged in to Hugging Face!")

✅ Logged in to Hugging Face!


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "google/medgemma-4b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [5]:
# Caminho do dataset
dataset_path = '/kaggle/input/datasets/richardsonsouza/dataset-prompts-v2/dataset_prompts.jsonl'

print(f"INICIANDO PROCESSAMENTO EM LOTE DO ARQUIVO: {dataset_path}\n")

# Configurações de Geração (Ajuste conforme necessidade)
gen_config = {
    "max_new_tokens": 1024,
    "temperature": 0.2,
    "do_sample": True,
    "top_p": 0.95
}

# Controle para teste (para não rodar o arquivo todo se for gigante)
# Defina como None para rodar tudo: LIMITE_TESTE = None
LIMITE_TESTE = 6
contador = 0

try:
    with open(dataset_path, 'r', encoding='utf-8') as f:
        for linha in f:
            # 1. Parada de segurança para testes
            if LIMITE_TESTE and contador >= LIMITE_TESTE:
                print(f"Limite de teste atingido ({LIMITE_TESTE} casos). Parando...")
                break
                
            # 2. Parse do JSON
            caso = json.loads(linha)
            
            # 3. Extração de Metadados (Direto do JSON)
            meta = caso.get('meta', {})
            municipio = meta.get('municipio', 'Desconhecido')
            data_alvo = meta.get('competencia', 'N/A')
            estacao = meta.get('estacao', 'N/A')
            
            # 4. Extração do Prompt
            prompt_texto = caso.get('prompt')
            
            if not prompt_texto:
                print(f"Pulo: Prompt vazio para ID {caso.get('id')}")
                continue

            # 5. Formatação para o Modelo
            chat = [
                { "role": "user", "content": prompt_texto },
            ]
            prompt_formatado = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
            
            # 6. Inferência
            inputs = tokenizer(prompt_formatado, return_tensors="pt").to("cuda")
            
            with torch.no_grad():
                outputs = model.generate(**inputs, **gen_config)
            
            # 7. Decodificação (Pegando apenas a resposta nova)
            tokens_gerados = outputs[0][inputs.input_ids.shape[-1]:]
            resposta_limpa = tokenizer.decode(tokens_gerados, skip_special_tokens=True)

            # 8. Exibição do Relatório
            print(f"CASO {contador + 1}: {municipio.upper()} ({data_alvo}) | Estação: {estacao}")
            print("-" * 80)
            print("RESPOSTA DO MEDGEMMA (JSON):")
            print(resposta_limpa)
            print("=" * 80 + "\n")
            
            # 9. Limpeza de Memória e Incremento
            del inputs, outputs
            torch.cuda.empty_cache()
            contador += 1

except FileNotFoundError:
    print("Erro: Arquivo não encontrado. Verifique o caminho.")
except Exception as e:
    print(f"Erro inesperado: {e}")

print("Processamento finalizado.")

INICIANDO PROCESSAMENTO EM LOTE DO ARQUIVO: /kaggle/input/datasets/richardsonsouza/dataset-prompts-v2/dataset_prompts.jsonl

CASO 1: BORBA (2025-11) | Estação: Vazante
--------------------------------------------------------------------------------
RESPOSTA DO MEDGEMMA (JSON):
```json
{
  "analise_situacional": "A estação Vazante, com rios em nível baixo e acesso difícil, representa um desafio para a realização do teste rápido treponêmico (Sífilis). A sazonalidade da sífilis, que pode aumentar em períodos de maior contato social, é um fator a ser considerado. O histórico recente de 90 dias sem internações ou diagnósticos relevantes sugere que a demanda por testes de sífilis pode ser relativamente estável, mas a dificuldade de acesso pode impactar a cobertura. A prioridade deve ser garantir a acessibilidade e a capacidade de realizar os testes, mesmo com recursos limitados. A baixa demanda, combinada com a dificuldade de acesso, torna o tratamento local a estratégia mais adequada.",
  "